# Tugas Prapraktikum

Tugas Prapraktikum dikerjakan dengan _dataset_ [Rain in Australia](https://www.kaggle.com/datasets/jsphyg/weather-dataset-rattle-package/download?datasetVersionNumber=2). Tanpa meninjau waktu (`date`), prediksi status hujan pada keesokan harinya (`RainTomorrow`). Berikan nilai `1` jika diprediksi hujan pada keesokan harinya, `0` jika tidak.

<br>
Tugas dikerjakan secara berkelompok. Setiap kelompok terdiri atas 2 (dua) mahasiswa. Kumpulkan paling lambat pada Minggu, 16 April 2023, pukul 23:59 WIB melalui Edunex.

# 0. Persiapan Data and Pustaka

## Install and import dependencies

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.model_selection import cross_validate, GridSearchCV
from sklearn.ensemble import VotingClassifier, StackingClassifier
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
import matplotlib.pyplot as plt

## Read the data

In [ ]:
df = pd.read_csv('weatherAUS.csv')
df = df.drop(['Date'], axis=1)
df.head()

# I. Pemahaman Data
Tujuan dari bagian ini adalah peserta dapat memahami kualitas dari data yang diberikan. Hal yang diliputi adalah sebagai berikut:
1. Ukuran data
2. Statistik dari tiap fitur
3. Pencilan (_outlier_)
4. Korelasi
5. Distribusi 

## I.1 
Carilah:
1. Ukuran dari data (instansi dan fitur)
2. Tipe dari setiap fitur 
3. Banyak nilai unik dari fitur yang bertipe kategorikal
4. Nilai minimum, maksimum, rata-rata, median, dan standar deviasi dari fitur nonkategorikal

In [ ]:
# I.1 Kode di sini.

# 1. Ukuran dari data (instansi dan fitur)
df.shape

In [ ]:
# 2. Tipe dari setiap fitur
df.dtypes

In [ ]:
# 3. Banyak nilai unik dari fitur yang bertipe kategorikal
df.select_dtypes(include=['object']).nunique()

In [ ]:
# 4. Nilai minimum, maksimum, rata-rata, median, dan standar deviasi dari fitur nonkategorikal
df.select_dtypes(include=['float64']).describe()

## I.2
Carilah:
1. Nilai hilang (_missing_) dari setiap fitur
2. Nilai pencilan (_outlier_) dari setiap fitur

In [ ]:
# I.2 Kode di sini.

# 1. Nilai hilang (missing) dari setiap fitur
df.isnull().sum()

In [ ]:
# 2. Nilai pencilan (outlier) dari setiap fitur

q1 = df.select_dtypes(include=['float64']).quantile(0.25)
q3 = df.select_dtypes(include=['float64']).quantile(0.75)
iqr = q3 - q1

minimum_range = q1 - 1.5 * iqr
maximum_range = q3 + 1.5 * iqr

# Nilai outlier cuman bisa di type number
((df.select_dtypes(include=['float64']) < minimum_range) | (df.select_dtypes(include=['float64']) > maximum_range)).sum()

## I.3
Lakukan:
1. Pencarian korelasi antarfitur
2. Visualisasi distribusi setiap fitur (kategorikal dan kontinu)
3. Visualisasi distribusi setiap fitur per target (`RainTomorrow`)

In [ ]:
# I.3 Kode di sini.

# 1. Pencarian korelasi antar fitur
print(df.select_dtypes(include=['float64']).corr())

In [ ]:
# 2. Visualisasi distribusi setiap fitur (kategorikal dan kontinu)

# Kontinu
df.hist(figsize=(20, 20))
plt.show()


In [ ]:
# Kategorikal
for col in df.select_dtypes(include=['object']):
    df[col].value_counts().plot(kind='bar')
    plt.title(col)
    plt.show()

## I.4
Lakukanlah analisis lebih lanjut jika diperlukan, kemudian lakukan hal berikut:
1. Penambahan fitur jika memungkinkan
2. Pembuangan fitur yang menurut kalian tidak dibutuhkan
3. Penanganan nilai hilang
4. Transformasi data kategorikal menjadi numerikal (_encoding_)
5. _Scaling_ dengan `MinMaxScaler`

In [ ]:
# I.4 Put your code here

# 1. Penambahan fitur jika memungkinkan

In [ ]:
# 2. Pembuangan fitur yang menurut kalian tidak dibutuhkan
df = df.drop(['Location', 'WindGustDir', 'WindDir9am', 'WindDir3pm', 'RainToday'], axis=1)

for col in df:
    if df[col].isnull().sum() > 10000:
        df = df.drop([col], axis=1)

print(df.shape)

In [ ]:
# 3. Penanganan nilai hilang
# Yang kategorikal pake modus
for col in df.select_dtypes(include=['object']).columns:
    df[col] = df[col].fillna(df[col].mode()[0])

# Yang numerikal pake mean
for col in df.select_dtypes(include=['float64']).columns:
    df[col] = df[col].fillna(df[col].mean())

In [ ]:
# 4. Transformasi data kategorikal menjadi numerikal (encoding)

encoder = LabelEncoder()
obj = df.select_dtypes(include=['object']).columns

for col in df.select_dtypes(include=['object']).columns:
    df[col] = encoder.fit_transform(df[col])

print(df)

In [ ]:
# 5. Scaling dengan MinMaxScaler

scaler = MinMaxScaler()
df_scaled = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)
print(df_scaled)

# II. Desain Eksperimen
Tujuan dari bagian ini adalah peserta dapat memahami cara melakukan eksperimen mencari metode terbaik dengan benar. Hal yang diliputi adalah sebagai berikut:
1. Pembuatan model
2. Proses validasi
3. _Hyperparameter tuning_

## II.1
Tentukanlah metrik yang akan digunakan pada eksperimen kali ini. Metrik yang dapat lebih dari satu jenis.

(Tuliskan jawaban bagian II.1 di sini.)

> Matriks yang akan digunakan adalah nilai akurasi, precision, recall, dan juga nilai F1

## II.2 
Bagi data dengan perbandingan 0,8 untuk data latih dan 0,2 untuk data validasi.

In [ ]:
# II.2 Kode di sini

X = df_scaled.drop(['RainTomorrow'], axis=1)
y = df_scaled['RainTomorrow']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)

## II.3
Lakukan hal berikut:
1. Prediksi dengan menggunakan model _logistic regression_ sebagai _baseline_.
2. Tampilkan evaluasi dari model yang dibangun dari metrik yang ditentukan pada II.1
3. Tampilkan _confusion matrix_.

In [ ]:
# II.3 Kode di sini

model = LogisticRegression(max_iter=10000).fit(X_train, y_train)
y_pred = model.predict(X_test)

print('Accuracy: ', accuracy_score(y_test, y_pred))
print('Precision: ', precision_score(y_test, y_pred))
print('Recall: ', recall_score(y_test, y_pred))
print('F1: ', f1_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

## II.4 
Lakukanlah:
1. Pembelajaran dengan model lain
2. _Hyperparameter tuning_ untuk model yang dipakai dengan menggunakan _grid search_ (perhatikan _random factor_ pada beberapa algoritma model)
3. Validasi dengan _cross validation_


In [ ]:
# II.4 Kode di sini.

# ndak tau modelnya yg mana, aing pake neural ae
svc = SVC()

param = {
    'C': [0.1, 1, 10],
    'gamma': [0.1, 1],
    'kernel': ['linear', 'rbf']
}

grid_search = GridSearchCV(svc, param, cv=5, scoring='accuracy').fit(X_train, y_train)
print(grid_search.best_params_)

svc2 = SVC(**grid_search.best_params_).fit(X_train, y_train)
y_pred = svc2.predict(X_test)

print('Accuracy: ', accuracy_score(y_test, y_pred))
print('Precision: ', precision_score(y_test, y_pred))
print('Recall: ', recall_score(y_test, y_pred))
print('F1: ', f1_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

In [ ]:
scores = cross_validate(svc, X_test, y_test, cv=5, scoring=('accuracy', 'precision', 'recall', 'f1'))

print('Accuracy: ', scores['test_accuracy'].mean())
print('Precision: ', scores['test_precision'].mean())
print('Recall: ', scores['test_recall'].mean())
print('F1: ', scores['test_f1'].mean())

# III. Improvement
Pada bagian ini, kalian diharapkan dapat:
1. melakukan pelatihan dengan data hasil _oversampling_ / _undersampling_, disertai dengan validasi yang benar; serta
2. menerapkan beberapa metode untuk menggabungkan beberapa model.

Kedua hal ini adalah contoh metode untuk meningkatkan kinerja dari model.

## III.1
Lakukanlah:
1. _Oversampling_ pada kelas minoritas pada data latih
2. _Undersampling_ pada kelas mayoritas pada data latih

Pada setiap tahap, latih dengan model *baseline* (II.3), dan validasi dengan data validasi. Data latih dan validasi adalah data yang disusun pada bagian II.2.

In [ ]:
# III.1 Kode di sini.

# Oversampling
smote = SMOTE(random_state=12)
OX_train, Oy_train = smote.fit_resample(X_train, y_train)
print(OX_train.shape)

model = LogisticRegression(max_iter=10000).fit(OX_train, Oy_train)
y_pred = model.predict(X_test)

print('Accuracy: ', accuracy_score(y_test, y_pred))
print('Precision: ', precision_score(y_test, y_pred))
print('Recall: ', recall_score(y_test, y_pred))
print('F1: ', f1_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

In [ ]:
# Validasi
scores = cross_validate(model, X_test, y_test, cv=5, scoring=('accuracy', 'precision', 'recall', 'f1'))

print('Accuracy: ', scores['test_accuracy'].mean())
print('Precision: ', scores['test_precision'].mean())
print('Recall: ', scores['test_recall'].mean())
print('F1: ', scores['test_f1'].mean())

In [ ]:
# Undersampling
rus = RandomUnderSampler(random_state=12)
UX_train, Uy_train = rus.fit_resample(X_train, y_train)
print(UX_train.shape)

model = LogisticRegression(max_iter=10000).fit(UX_train, Uy_train)
y_pred = model.predict(X_test)

print('Accuracy: ', accuracy_score(y_test, y_pred))
print('Precision: ', precision_score(y_test, y_pred))
print('Recall: ', recall_score(y_test, y_pred))
print('F1: ', f1_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

In [ ]:
scores = cross_validate(model, X_test, y_test, cv=5, scoring=('accuracy', 'precision', 'recall', 'f1'))

print('Accuracy: ', scores['test_accuracy'].mean())
print('Precision: ', scores['test_precision'].mean())
print('Recall: ', scores['test_recall'].mean())
print('F1: ', scores['test_f1'].mean())

## III.2
Lakukanlah:
1. Eksplorasi _soft voting_, _hard voting_, dan _stacking_.
2. Buatlah model _logistic regression_ dan SVM.
3. Lakukanlah _soft voting_ dari model-model yang dibangun pada poin 2.
4. Lakukan _hard voting_ dari model-model yang dibangun pada poin 2.
5. Lakukanlah _stacking_ dengan _final classifier_ adalah _logistic regression_ dari model-model yang dibangun pada poin 2.
6. Lakukan validasi dengan metrics yang telah ditentukan untuk poin 3, 4, dan 5.

(Tuliskan hasil eksplorasi III.2 poin 1 di sini.)

In [ ]:
# III.2 Kode di sini.
lr = LogisticRegression(max_iter=1000)
svm = SVC(probability=True)

In [ ]:
# Soft Voting
model = VotingClassifier(estimators=[('lr', lr), ('svm', svm)], voting='soft')
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print('Accuracy: ', accuracy_score(y_test, y_pred))
print('Precision: ', precision_score(y_test, y_pred))
print('Recall: ', recall_score(y_test, y_pred))
print('F1: ', f1_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

scores = cross_validate(model, X_test, y_test, cv=5, scoring=('accuracy', 'precision', 'recall', 'f1'))

print('Accuracy: ', scores['test_accuracy'].mean())
print('Precision: ', scores['test_precision'].mean())
print('Recall: ', scores['test_recall'].mean())
print('F1: ', scores['test_f1'].mean())

In [ ]:
# Hard Voting
model = VotingClassifier(estimators=[('lr', lr), ('svm', svm)], voting='hard')
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print('Accuracy: ', accuracy_score(y_test, y_pred))
print('Precision: ', precision_score(y_test, y_pred))
print('Recall: ', recall_score(y_test, y_pred))
print('F1: ', f1_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

scores = cross_validate(model, X_test, y_test, cv=5, scoring=('accuracy', 'precision', 'recall', 'f1'))

print('Accuracy: ', scores['test_accuracy'].mean())
print('Precision: ', scores['test_precision'].mean())
print('Recall: ', scores['test_recall'].mean())
print('F1: ', scores['test_f1'].mean())

In [ ]:
model = StackingClassifier(estimators=[('lr', lr), ('svm', svm)], final_estimator=LogisticRegression(max_iter=10000))
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print('Accuracy: ', accuracy_score(y_test, y_pred))
print('Precision: ', precision_score(y_test, y_pred))
print('Recall: ', recall_score(y_test, y_pred))
print('F1: ', f1_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

scores = cross_validate(model, X_test, y_test, cv=5, scoring=('accuracy', 'precision', 'recall', 'f1'))

print('Accuracy: ', scores['test_accuracy'].mean())
print('Precision: ', scores['test_precision'].mean())
print('Recall: ', scores['test_recall'].mean())
print('F1: ', scores['test_f1'].mean())

# IV. Analisis
Bandingkan hasil dari hal-hal berikut:
1. Model _baseline_ (II.3)
2. Model lain (II.4)
3. Hasil _undersampling_
4. Hasil _oversampling_
5. Hasil _soft voting_
6. Hasil _hard voting_
7. Hasil _stacking_

(Tuliskan jawaban bagian IV di sini.)

## IV.1

## IV.2

## IV.4

## IV.5

## IV.6

## IV.7
